In [ ]:
src_path = "/home/hulabdl/exp/TaskR1_h5"
tgt_path = "/home/hulabdl/exp/taskr1out/bart-fista"

from glob import glob
import os
import h5py
import numpy as np
from einops import rearrange
import sys
os.environ["BART_TOOLBOX_PATH"] = "/home/hulabdl/bart-installer/bart"
sys.path.append(os.path.join("/home/hulabdl/bart-installer/bart", 'python'))
from bart import bart

from pqdm.processes import pqdm

import torch


def recon(fname):
    with h5py.File(fname) as file:
        kus = file["kus"]
        kus = kus['real'] + 1j * kus['imag']
        if len(kus.shape) == 4:
            kus = rearrange(kus, "slice coil radout phase -> readout phase 1 coil 1 1 1 1 1 1 1 1 1 slice 1 1")
            csm = bart(1, "ecalib -m 1 -d0", kus)
            recon = bart(1, "pics -l1 -d0 -r0.001", kus, csm)
            while len(recon.shape) < 16:
                recon = np.expand_dims(recon, axis=-1)
            recon = rearrange(recon, "readout phase 1 1 1 1 1 1 1 1 1 1 1 slice 1 1 -> slice readout phase")

            recon = torch.as_tensor(recon)

            torch.save(recon, os.path.join(tgt_path, os.path.basename(fname).replace(".h5", ".pt")))

        elif len(kus.shape) == 5:
            kus = rearrange(kus, "time slice coil readout phase -> readout phase 1 coil 1 1 1 1 1 1 time 1 1 slice 1 1")
            csm = bart(1, "ecalib -m 1 -d0 ", kus)
            recon = bart(1, "pics -l1 -d0 -r0.001", kus, csm)
            while len(recon.shape) < 16: 
                recon = np.expand_dims(recon, axis=-1)

            recon = rearrange(recon, "readout phase 1 1 1 1 1 1 1 1 time 1 1 slice 1 1 -> time slice readout phase")
            recon = torch.as_tensor(recon)
            torch.save(recon, os.path.join(tgt_path, os.path.basename(fname).replace(".h5", ".pt")))

        else:
            raise ValueError("")

fnamelist = list(glob(os.path.join(src_path, "*.h5")))

pqdm(fnamelist, recon, n_jobs=2, desc="Reconstructing data")

In [ ]:
# Retry
from glob import glob
import os

src_path = "/home/hulabdl/exp/TaskR2_h5"
finished_path = "/home/hulabdl/exp/taskr2out/bart-fista"

src_fnamelist = list(glob(os.path.join(src_path, "*.h5")))
src_basenamelist = [os.path.basename(f) for f in src_fnamelist]

finished_fnamelist = list(glob(os.path.join(finished_path, "*.pt")))
finished_basenamelist = [os.path.basename(f) for f in finished_fnamelist]

unfinished_basenamelist = [f for f in src_basenamelist if f.replace(".h5", ".pt") not in finished_basenamelist]
src_unfinished_fnamelist = [os.path.join(src_path, f) for f in unfinished_basenamelist]

# src_path = src_unfinished_fnamelist
tgt_path = "/home/hulabdl/exp/taskr2out/bart-fista"

from glob import glob
import os
import h5py
import numpy as np
from einops import rearrange
import sys
os.environ["BART_TOOLBOX_PATH"] = "/home/hulabdl/bart-installer/bart"
sys.path.append(os.path.join("/home/hulabdl/bart-installer/bart", 'python'))
from bart import bart

from pqdm.processes import pqdm

import torch


def recon(fname):
    with h5py.File(fname) as file:
        kus = file["kus"]
        kus = kus['real'] + 1j * kus['imag']
        if len(kus.shape) == 4:
            kus = rearrange(kus, "slice coil readout phase -> readout phase 1 coil 1 1 1 1 1 1 1 1 1 slice 1 1")
            csm = bart(1, "ecalib -m 1 -d0", kus)
            recon = bart(1, "pics -l1 -d0 -r0.001", kus, csm)
            while len(recon.shape) < 16:
                recon = np.expand_dims(recon, axis=-1)
            recon = rearrange(recon, "readout phase 1 1 1 1 1 1 1 1 1 1 1 slice 1 1 -> slice readout phase")

            recon = torch.as_tensor(recon)

            torch.save(recon, os.path.join(tgt_path, os.path.basename(fname).replace(".h5", ".pt")))

        elif len(kus.shape) == 5:
            kus = rearrange(kus, "time slice coil readout phase -> readout phase 1 coil 1 1 1 1 1 1 time 1 1 slice 1 1")
            csm = bart(1, "ecalib -m 1 -d0 ", kus)
            recon = bart(1, "pics -l1 -d0 -r0.001", kus, csm)
            while len(recon.shape) < 16: 
                recon = np.expand_dims(recon, axis=-1)

            recon = rearrange(recon, "readout phase 1 1 1 1 1 1 1 1 time 1 1 slice 1 1 -> time slice readout phase")
            recon = torch.as_tensor(recon)
            torch.save(recon, os.path.join(tgt_path, os.path.basename(fname).replace(".h5", ".pt")))

        else:
            raise ValueError("")

fnamelist = src_unfinished_fnamelist

# pqdm(fnamelist, recon, n_jobs=2, desc="Reconstructing data")

for fname in fnamelist:
    recon(fname)
